## 🔒 LABOR MARKET TIGHTNESS

**Indicators:**

| Indicator                  | Source              | Notes                                                             |
| -------------------------- | ------------------- | ----------------------------------------------------------------- |
| Job Openings Rate          | JOLTS (`JTSJOL`)    | Total Nonfarm or by industry                                      |
| Hires Rate                 | JOLTS (`JTSHIR`)    | Hiring flow                                                       |
| Beveridge Ratio            | Derived             | = Job Openings / Unemployed                                       |
| Openings per Hire          | Derived             | = Job Openings / Hires                                            |
| **Prime-Age LFPR (25–54)** | BLS (`LNS11300060`) | Best signal of active labor force engagement, avoids demographics |

**Feature Engineering Ideas:**

* `OpeningsPerUnemployed = Job Openings / Unemployed`
* `OpeningsPerHire = Job Openings / Hires`
* `PrimeLFPR_Zscore = z-score(LNS11300060)`
* `ParticipationGap = Pre-pandemic Prime LFPR – Current Prime LFPR`
* Normalize tightness indicators using z-scores or rolling trends


In [2]:
import pandas as pd

LMT_df= pd.read_excel('Data/Labor_Market_Tightness.xlsx')

# FEATURE ENGINEERING

## Calculate the Openings_Per_Unemployed
LMT_df["Openings_Per_Unemployed"]= LMT_df['Job_Openings_Rate']/LMT_df['Unemployment_Level']

## Calculate the Openings_Per_Hire
LMT_df["Openings_Per_Hire"]= LMT_df['Job_Openings_Rate']/LMT_df['Hires_Rate']

## Calculate the PrimeLFPR Z-Score
LMT_df["PrimeLFPR_Zscore"]= (LMT_df['Prime_Age_Labor_Force_Participation_Rate'] - LMT_df['Prime_Age_Labor_Force_Participation_Rate'].mean()) / LMT_df['Prime_Age_Labor_Force_Participation_Rate'].std()

## Calculate the ParticipationGap

baseline = LMT_df[(LMT_df['observation_date'] >= '2015-01-01') & (LMT_df['observation_date'] <= '2019-12-01')]['Prime_Age_Labor_Force_Participation_Rate'].mean()

LMT_df['ParticipationGap'] = baseline - LMT_df['Prime_Age_Labor_Force_Participation_Rate']

LMT_df


,observation_date,Job_Openings_Rate,Hires_Rate,Prime_Age_Labor_Force_Participation_Rate,Unemployment_Level,Openings_Per_Unemployed,Openings_Per_Hire,PrimeLFPR_Zscore,ParticipationGap
0,2000-12-01,5088.0,5426.0,-0.11905,5634,0.903088,0.937707,-0.144961,0.507885
1,2001-01-01,5234.0,5722.0,-0.35545,6023,0.869002,0.914715,-0.488144,0.744285
2,2001-02-01,5097.0,5303.0,-0.35545,6089,0.837083,0.961154,-0.488144,0.744285
3,2001-03-01,4762.0,5528.0,-0.23725,6141,0.775444,0.861433,-0.316553,0.626085
4,2001-04-01,4615.0,5204.0,-0.82938,6271,0.735927,0.886818,-1.176151,1.218215
...,...,...,...,...,...,...,...,...,...
288,2024-12-01,7508.0,5374.0,0.24038,6886,1.090328,1.397097,0.376825,0.148455
289,2025-01-01,7762.0,5371.0,0.24010,6849,1.133304,1.445168,0.376418,0.148735
290,2025-02-01,7480.0,5370.0,0.00000,7052,1.060692,1.392924,0.027864,0.388835
291,2025-03-01,7192.0,5411.0,-0.11990,7083,1.015389,1.329144,-0.146195,0.508735


## 😣 LABOR MARKET DISTRESS

**Indicators:**

| Indicator                 | Source              | Notes                                        |
| ------------------------- | ------------------- | -------------------------------------------- |
| Unemployment Rate (U-3)   | BLS (`UNRATE`)      | Official unemployment rate                   |
| **U-6 Unemployment Rate** | BLS (`U6RATE`)      | Includes underemployed & discouraged workers |
| Initial Jobless Claims    | FRED (`ICSA`)       | Weekly job loss inflow                       |
| Continued Jobless Claims  | FRED (`CCSA`)       | Persistent unemployment inflow               |
| Median Weeks Unemployed   | BLS (`LNS13008275`) | Duration signal of unemployment hardship     |
| Long-term Unemployed (%)  | Derived             | = UEMP27OV / UNEMPLOY                        |

**Feature Engineering Ideas:**

* `Unemployed27Share = UEMP27OV / UNEMPLOY`
* `Lagged_Claims = Rolling average of ICSA over 4–8 weeks`
* `MedianWeeks_Zscore`
* `U6_U3_Spread = U6RATE - UNRATE` (hidden slack signal)
* Combine into a `DistressIndex` using PCA or averaging

In [3]:
import pandas as pd

LMD_df= pd.read_excel('Data/Labor_Market_Distress.xlsx')

# FEATURE ENGINEERING

## Calculate the Long-Term Unemployed (%)
LMD_df["%_Long_Term_Unemployed"]= LMD_df['Unemployed_27_Weeks_More']/LMD_df['Unemployment_Level']

## Calculate the Lagged_Claims (Rolling average of ICSA over 4-8 Weeks)
Lagged_Claims_4m = LMD_df['Initial_Jobless_Claims'].rolling(window=4).mean()
Lagged_Claims_8m = LMD_df['Initial_Jobless_Claims'].rolling(window=8).mean()

LMD_df['Lagged_Claims_Average'] = (Lagged_Claims_4m + Lagged_Claims_8m) / 2

## Calculate Median Weeks Z-Score
LMD_df["MedianWeeks_Zscore"]= (LMD_df['Median_Weeks_Unemployed'] - LMD_df['Median_Weeks_Unemployed'].mean()) / LMD_df['Median_Weeks_Unemployed'].std()

## Calculate U6_U3_Spread (hidden slack signal)
LMD_df["U6_U3_Spread"]=  LMD_df['%_U6_Unemployment_Rate'] - LMD_df['%_U3_Unemployment_Rate']

LMD_df


,observation_date,%_U3_Unemployment_Rate,%_U6_Unemployment_Rate,Initial_Jobless_Claims,Continued_Jobless_Claims,Median_Weeks_Unemployed,Unemployment_Level,Unemployed_27_Weeks_More,%_Long_Term_Unemployed,Lagged_Claims_Average,MedianWeeks_Zscore,U6_U3_Spread
0,2000-12-01,3.9,6.9,346000,2306400,6.0,5634,642,0.113951,NaN,-1.326004,3.0
1,2001-01-01,4.2,7.3,340000,2395750,5.8,6023,676,0.112236,NaN,-1.370951,3.1
2,2001-02-01,4.2,7.3,371250,2486500,6.1,6089,714,0.117261,NaN,-1.303531,3.1
3,2001-03-01,4.3,7.3,387200,2585400,6.6,6141,696,0.113337,NaN,-1.191164,3.0
4,2001-04-01,4.4,7.4,396750,2697250,5.9,6271,712,0.113539,NaN,-1.348478,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
288,2024-12-01,4.1,7.5,222250,1860750,10.4,6886,1551,0.225240,227100.00,-0.337177,3.4
289,2025-01-01,4.0,7.5,213500,1865250,10.4,6849,1443,0.210688,225131.25,-0.337177,3.5
290,2025-02-01,4.1,8.0,226000,1859750,10.0,7052,1455,0.206324,223250.00,-0.427070,3.9
291,2025-03-01,4.2,7.9,223200,1863200,9.8,7083,1495,0.211069,222893.75,-0.472017,3.7


## 🔁 LABOR MOBILITY / CONFIDENCE

**Indicators:**

| Indicator              | Source           | Notes                                       |
| ---------------------- | ---------------- | ------------------------------------------- |
| Quit Rate              | JOLTS (`JTSQUR`) | Lower values suggest fear of switching jobs |
| Quits-to-Layoffs Ratio | Derived          | = Quits / Layoffs                           |
| Layoffs and Discharges | JOLTS (`JTSLDL`) | Indicator of involuntary exits              |

**Feature Engineering Ideas:**

* `QuitsToLayoffs = Quits / Layoffs`
* `LayoffShock = % change in Layoffs MoM`
* Normalize using rolling z-scores or trend slopes

In [4]:
import pandas as pd

LMCM_df = pd.read_excel('Data/Labor_Confidence_and_Mobility.xlsx')

# FEATURE ENGINEERING

## Calculate the Quits-to-Layoff_Ratio
LMCM_df['Quits_to_Layoffs']= LMCM_df['Quits_Rate']/LMCM_df['Layoffs_and_Discharges']

## Calculate the LayoffShock

LMCM_df['LayoffShock']= LMCM_df['Layoffs_and_Discharges'].pct_change() * 100

LMCM_df

,observation_date,Quits_Rate,Layoffs_and_Discharges,Quits_to_Layoffs,LayoffShock
0,2000-12-01,2882,2018,1.428147,NaN
1,2001-01-01,3245,2220,1.461712,10.009911
2,2001-02-01,3053,1855,1.645822,-16.441441
3,2001-03-01,3054,2133,1.431786,14.986523
4,2001-04-01,3163,1883,1.679766,-11.720581
...,...,...,...,...,...
287,2024-11-01,3032,1739,1.743531,-2.740492
288,2024-12-01,3095,1669,1.854404,-4.025302
289,2025-01-01,3256,1674,1.945042,0.299581
290,2025-02-01,3250,1780,1.825843,6.332139


## 💸 COMPENSATION & PARTICIPATION SIGNALS

**Indicators:**

| Indicator                                           | Source                | Notes                                                  |
| --------------------------------------------------- | --------------------- | ------------------------------------------------------ |
| AHE: Production & Nonsupervisory (Total Private)    | CES (`CES0500000030`) | Nominal wages for majority of working population       |
| Prime-Age Employment-to-Population Ratio            | BLS (`LNS12300060`)   | Measures active employment engagement for 25–54 y/o    |
| **Prime-Age Labor Force Participation Rate (LFPR)** | BLS (`LNS11300060`)   | Cleanest signal of labor market participation strength |
| CPI Inflation (YoY % Change)                        | FRED (`CPIAUCSL`)     | Used to adjust AHE into real wage growth               |

**Feature Engineering Ideas:**

* `AHE_YoY = df['AHE'].pct_change(periods=12) * 100`
* `RealWageGrowth = AHE_YoY - CPI_YoY`
* `WageGrowth_YoY = AHE_YoY`
* `PrimeAgeEPOP_Zscore`
* `PrimeLFPR_Trend = df['PrimeLFPR'] - df['PrimeLFPR'].shift(12)`
* Combine into a `ParticipationIndex` if desired




In [5]:
CPS_df = pd.read_excel('Data/Compensation_and_Participation_Signal.xlsx')
# FEATURE ENGINEERING

## Calculate Average_Weekly_Earning Year-to-Year
CPS_df['AWE_YoY'] = CPS_df['Average_Weekly_Earnings'].pct_change(periods=12) * 100

## Calculate RealWageGrowth
CPI_YoY = CPS_df['Consumer_Price_Index'].pct_change(periods=12) * 100
CPS_df['RealWageGrowth'] = CPS_df['AWE_YoY'] - CPI_YoY

## Calculate PrimeAgeEPOP Z-score
CPS_df["Prime_Age_Employment_to_Population_Ratio"]= (CPS_df['Prime_Age_Employment_to_Population_Ratio'] - CPS_df['Prime_Age_Employment_to_Population_Ratio'].mean()) / CPS_df['Prime_Age_Employment_to_Population_Ratio'].std()

## Calculate PrimeLFPR_Trend
CPS_df["PrimeLFPR_Trend"]= CPS_df['Prime_Age_Labor_Force_Participation_Rate'] - CPS_df['Prime_Age_Labor_Force_Participation_Rate'].shift(12)

CPS_df

,observation_date,Average_Weekly_Earnings,Prime_Age_Employment_to_Population_Ratio,Consumer_Price_Index,Prime_Age_Labor_Force_Participation_Rate,AWE_YoY,RealWageGrowth,PrimeLFPR_Trend
0,2000-12-01,485.86,1.557256,174.600,-0.11905,NaN,NaN,NaN
1,2001-01-01,488.72,1.557256,175.600,-0.35545,NaN,NaN,NaN
2,2001-02-01,488.24,1.507319,176.000,-0.35545,NaN,NaN,NaN
3,2001-03-01,491.72,1.507319,176.100,-0.23725,NaN,NaN,NaN
4,2001-04-01,490.96,1.307570,176.400,-0.82938,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
288,2024-12-01,1033.58,1.107821,317.603,0.24038,3.729351,0.856985,-0.48601
289,2025-01-01,1034.88,1.207696,319.086,0.24010,4.328891,1.329478,-0.36376
290,2025-02-01,1038.58,1.107821,319.775,0.00000,3.870464,1.056194,-0.60241
291,2025-03-01,1046.45,1.057884,319.615,-0.11990,3.892816,1.487231,-0.48091


## 🧊 COMPOSITE INDEX FEATURE ENGINEERING (Frozen Index)

To build the **Frozen Job Market Index**, follow this workflow:

### 1. **Standardize (Z-score)** the following variables:

| Direction        | Variable                                 |
| ---------------- | ---------------------------------------- |
| ⬆️ = More Frozen | Median Weeks Unemployed                  |
| ⬆️ = More Frozen | Unemployment Rate (U-3)                  |
| ⬆️ = More Frozen | Initial Jobless Claims                   |
| ⬆️ = More Frozen | U6–U3 Spread                             |
| ⬇️ = More Frozen | Beveridge Ratio / OpeningsPerUnemployed  |
| ⬇️ = More Frozen | Quit Rate                                |
| ⬇️ = More Frozen | Quits-to-Layoffs Ratio                   |
| ⬇️ = More Frozen | Prime-Age Labor Force Participation Rate |
| ⬇️ = More Frozen | Prime-Age Employment-to-Population Ratio |
| ⬇️ = More Frozen | Real Wage Growth                         

In [ ]:
import pandas as pd
import sqlite3


# Set up in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Load data into SQL tables
CPS_df.to_sql('CPS', conn, index=False, if_exists='replace')
LMCM_df.to_sql('LMCM', conn, index=False, if_exists='replace')
LMD_df.to_sql('LMD', conn, index=False, if_exists='replace')
LMT_df.to_sql('LMT', conn, index=False, if_exists='replace')



# SQL query to join all lagging indicator tables on observation_date
sql_query = """
SELECT Median_Weeks_Unemployed, 
FROM CPS
LEFT JOIN LMCM ON CPS.observation_date = LMCM.observation_date
LEFT JOIN LMD ON CPS.observation_date = LMD.observation_date
LEFT JOIN LMT ON CPS.observation_date = LMT.observation_date
"""

# Execute query and load result into DataFrame
Full_Dataset_df = pd.read_sql_query(sql_query, conn)

# Save to CSV

# Close the database connection
conn.close()

# Display the final joined DataFrame
Full_Dataset_df.columns


Index(['observation_date', 'Average_Weekly_Earnings',
       'Prime_Age_Employment_to_Population_Ratio', 'Consumer_Price_Index',
       'Prime_Age_Labor_Force_Participation_Rate', 'AWE_YoY', 'RealWageGrowth',
       'PrimeLFPR_Trend', 'observation_date', 'Quits_Rate',
       'Layoffs_and_Discharges', 'Quits_to_Layoffs', 'LayoffShock',
       'observation_date', '%_U3_Unemployment_Rate', '%_U6_Unemployment_Rate',
       'Initial_Jobless_Claims', 'Continued_Jobless_Claims',
       'Median_Weeks_Unemployed', 'Unemployment_Level',
       'Unemployed_27_Weeks_More', '%_Long_Term_Unemployed',
       'Lagged_Claims_Average', 'MedianWeeks_Zscore', 'U6_U3_Spread',
       'observation_date', 'Job_Openings_Rate', 'Hires_Rate',
       'Prime_Age_Labor_Force_Participation_Rate', 'Unemployment_Level',
       'Openings_Per_Unemployed', 'Openings_Per_Hire', 'PrimeLFPR_Zscore',
       'ParticipationGap'],
      dtype='object')